# Naive Long Short Term Memory Unit Implementation
This Notebook contains the code written to implement the NaiveLSTM unit for NLP/DL text applications.

In [17]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.optim as optim

from typing import *
from pathlib import Path

from enum import IntEnum

class Dim(IntEnum):
    batch = 0
    seq = 1
    feature = 2
    
class NaiveLSTM(nn.Module):
    def __init__(self, input_sz: int, hidden_sz: int):
        super().__init__()
        self.input_size = input_sz
        self.hidden_size = hidden_sz
        # input gate
        self.W_ii = Parameter(torch.Tensor(input_sz, hidden_sz))
        self.W_hi = Parameter(torch.Tensor(hidden_sz, hidden_sz))
        self.b_i = Parameter(torch.Tensor(hidden_sz))
        # forget gate
        self.W_if = Parameter(torch.Tensor(input_sz, hidden_sz))
        self.W_hf = Parameter(torch.Tensor(hidden_sz, hidden_sz))
        self.b_f = Parameter(torch.Tensor(hidden_sz))
        # ???
        self.W_ig = Parameter(torch.Tensor(input_sz, hidden_sz))
        self.W_hg = Parameter(torch.Tensor(hidden_sz, hidden_sz))
        self.b_g = Parameter(torch.Tensor(hidden_sz))
        # output gate
        self.W_io = Parameter(torch.Tensor(input_sz, hidden_sz))
        self.W_ho = Parameter(torch.Tensor(hidden_sz, hidden_sz))
        self.b_o = Parameter(torch.Tensor(hidden_sz))
        
        self.init_weights()
    
    
    def init_weights(self):
        for p in self.parameters():
            if p.data.ndimension() >= 2:
                nn.init.xavier_uniform_(p.data)
            else:
                nn.init.zeros_(p.data)
        
    def forward(self, x: torch.Tensor, 
                init_states: Optional[Tuple[torch.Tensor, torch.Tensor]]=None
               ) -> Tuple[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        """Assumes x is of shape (batch, sequence, feature)"""
        bs, seq_sz, _ = x.size()
        hidden_seq = []
        if init_states is None:
            h_t, c_t = torch.zeros(self.hidden_size).to(x.device), torch.zeros(self.hidden_size).to(x.device)
        else:
            h_t, c_t = init_states
        for t in range(seq_sz): # iterate over the time steps
            x_t = x[:, t, :]
            i_t = torch.sigmoid(x_t @ self.W_ii + h_t @ self.W_hi + self.b_i)
            f_t = torch.sigmoid(x_t @ self.W_if + h_t @ self.W_hf + self.b_f)
            g_t = torch.tanh(x_t @ self.W_ig + h_t @ self.W_hg + self.b_g)
            o_t = torch.sigmoid(x_t @ self.W_io + h_t @ self.W_ho + self.b_o)
            c_t = f_t * c_t + i_t * g_t
            h_t = o_t * torch.tanh(c_t)
            hidden_seq.append(h_t.unsqueeze(Dim.batch))
        hidden_seq = torch.cat(hidden_seq, dim=Dim.batch)
        # reshape from shape (sequence, batch, feature) to (batch, sequence, feature)
        hidden_seq = hidden_seq.transpose(Dim.batch, Dim.seq).contiguous()
        return hidden_seq, (h_t, c_t)
    
# [1] http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf

In [18]:
bs, seq_len, feat_sz, hidden_sz = 5, 10, 32, 16
arr = torch.randn(bs, seq_len, feat_sz)
lstm = NaiveLSTM(feat_sz, hidden_sz)

In [19]:
hs, (hn, cn) = lstm(arr)

In [20]:
hs.shape

torch.Size([5, 10, 16])

In [21]:
!curl http://www.sls.hawaii.edu/bley-vroman/brown.txt -o {"brown.txt"}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  1 6040k    1  103k    0     0  66259      0  0:01:33  0:00:01  0:01:32 66259
 14 6040k   14  892k    0     0   351k      0  0:00:17  0:00:02  0:00:15  351k
 66 6040k   66 4021k    0     0  1119k      0  0:00:05  0:00:03  0:00:02 1119k
100 6040k  100 6040k    0     0  1674k      0  0:00:03  0:00:03 --:--:-- 1674k


In [22]:
!pip install allennlp

  Stored in directory: C:\Users\NunYurBusiness\AppData\Local\pip\Cache\wheels\51\30\d1\92a39ba40f21cb70e53f8af96eb98f002a781843c065406500
  Stored in directory: C:\Users\NunYurBusiness\AppData\Local\pip\Cache\wheels\8d\52\86\e5a83b1797e7d263b458d2334edd2704c78508b3eea9323718
  Stored in directory: C:\Users\NunYurBusiness\AppData\Local\pip\Cache\wheels\b7\8d\e7\a0e74217da5caeb3c1c7689639b6d28ddbf9985b840bc96a9a
  Stored in directory: C:\Users\NunYurBusiness\AppData\Local\pip\Cache\wheels\46\2f\53\5f5c1d275492f2fce1cdab9a9bb12d49286dead829a4078e0e
  Stored in directory: C:\Users\NunYurBusiness\AppData\Local\pip\Cache\wheels\43\34\ce\cbb38d71543c408de56f3c5e26ce8ba495a0fa5a28eaaf1046
  Stored in directory: C:\Users\NunYurBusiness\AppData\Local\pip\Cache\wheels\ae\9b\66\6b8f275ec6dfa2138a56faaf2b4314ca44a4746286560d5578
Successfully built numpydoc overrides parsimonious word2number ftfy snowballstemmer
